## Inspect Results 

In [39]:
import os
import json

def count_experiment_stats(users_dict_path, target_run_id=None):
    """
    Counts basic stats for a given run_id across all user files.
    
    :param users_dict_path: Path to directory containing user .jsonl files
    :param target_run_id: Optional run_id to analyze. If None, uses latest from first file
    :return: Dictionary with counts and statistics
    """
    
    # Find all jsonl files
    user_files = [f for f in os.listdir(users_dict_path) if f.endswith('.jsonl')]
    if not user_files:
        print(f"No .jsonl files found in {users_dict_path}")
        return None
    
    total_files = len(user_files)
    files_with_run_data = 0
    
    # Get run_id if not provided
    if target_run_id is None:
        for uf in user_files:
            try:
                with open(os.path.join(users_dict_path, uf), 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    if len(lines) > 2:
                        runs_data = json.loads(lines[2].strip())
                        if runs_data.get('runs'):
                            target_run_id = runs_data['runs'][-1]['run_id']
                            print(f"Using run_id from {uf}: {target_run_id}")
                            break
            except (json.JSONDecodeError, IndexError, FileNotFoundError):
                continue
    
    if not target_run_id:
        print("Could not determine run_id")
        return None
    
    # Count stats
    user_stats = {}
    
    for user_file in user_files:
        user_file_path = os.path.join(users_dict_path, user_file)
        user_id = os.path.splitext(user_file)[0]
        
        try:
            with open(user_file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            
            # Get actual user_id from file if available
            if len(lines) > 0:
                user_info = json.loads(lines[0].strip())
                if 'user_id' in user_info:
                    user_id = user_info['user_id']
            
            # Check if this file has run data for our target run
            has_run_data = False
            imitation_count = 0
            
            if len(lines) > 2:
                try:
                    runs_content = json.loads(lines[2].strip())
                    target_run = next((r for r in runs_content.get('runs', []) 
                                     if r.get('run_id') == target_run_id), None)
                    if target_run:
                        has_run_data = True
                        imitation_count = len(target_run.get('imitations', []))
                except json.JSONDecodeError:
                    pass
            
            if has_run_data:
                files_with_run_data += 1
                
                # Count rounds (evaluations)
                round_count = 0
                if len(lines) > 3:
                    try:
                        eval_content = json.loads(lines[3].strip())
                        round_count = len([e for e in eval_content.get('evaluations', []) 
                                         if e.get('run_id') == target_run_id])
                    except json.JSONDecodeError:
                        pass
                
                user_stats[user_id] = {
                    'rounds': round_count,
                    'imitations': imitation_count
                }
        
        except Exception as e:
            print(f"Error processing {user_file}: {e}")
            continue
    
    # Summary
    results = {
        'run_id': target_run_id,
        'total_files_checked': total_files,
        'files_with_run_data': files_with_run_data,
        'total_users_in_run': len(user_stats),
        'user_stats': user_stats
    }
    
    return results

def print_summary(results):
    """Print a nice summary of the results"""
    if not results:
        print("No results to display")
        return
    
    print(f"\n=== EXPERIMENT SUMMARY ===")
    print(f"Run ID: {results['run_id']}")
    print(f"Files checked: {results['total_files_checked']}")
    print(f"Files with run data: {results['files_with_run_data']}")
    print(f"Users in this run: {results['total_users_in_run']}")
    
    if results['user_stats']:
        print(f"\nPer-user breakdown:")
        for user_id, stats in results['user_stats'].items():
            print(f"  {user_id}: {stats['rounds']} rounds, {stats['imitations']} imitations")
        
        # Quick stats
        rounds_list = [stats['rounds'] for stats in results['user_stats'].values()]
        imitations_list = [stats['imitations'] for stats in results['user_stats'].values()]
        
        print(f"\nQuick stats:")
        print(f"  Total rounds across all users: {sum(rounds_list)}")
        print(f"  Total imitations across all users: {sum(imitations_list)}")
        if rounds_list:
            print(f"  Average rounds per user: {sum(rounds_list)/len(rounds_list):.1f}")
        if imitations_list:
            print(f"  Average imitations per user: {sum(imitations_list)/len(imitations_list):.1f}")

# Usage example:
if __name__ == "__main__":
    # Replace with your actual path
    path = r"data\filtered_users"
    
    # Count stats for latest run (or specify run_id)
    results = count_experiment_stats(path, '20250708_125947')
    
    if results:
        print_summary(results)


=== EXPERIMENT SUMMARY ===
Run ID: 20250708_125947
Files checked: 161
Files with run data: 3
Users in this run: 3

Per-user breakdown:
  1.5196466670453228e+18: 1 rounds, 9 imitations
  27995424.0: 1 rounds, 9 imitations
  534023.0: 1 rounds, 9 imitations

Quick stats:
  Total rounds across all users: 3
  Total imitations across all users: 27
  Average rounds per user: 1.0
  Average imitations per user: 9.0


In [40]:
import os
import json

def count_experiment_stats_fixed(users_dict_path, target_run_id=None):
    """
    Counts basic stats for a given run_id across all user files.
    FIXED VERSION: Properly counts unique rounds instead of total evaluations
    
    :param users_dict_path: Path to directory containing user .jsonl files
    :param target_run_id: Optional run_id to analyze. If None, uses latest from first file
    :return: Dictionary with counts and statistics
    """
    
    # Find all jsonl files
    user_files = [f for f in os.listdir(users_dict_path) if f.endswith('.jsonl')]
    if not user_files:
        print(f"No .jsonl files found in {users_dict_path}")
        return None
    
    total_files = len(user_files)
    files_with_run_data = 0
    
    # Get run_id if not provided
    if target_run_id is None:
        for uf in user_files:
            try:
                with open(os.path.join(users_dict_path, uf), 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    if len(lines) > 2:
                        runs_data = json.loads(lines[2].strip())
                        if runs_data.get('runs'):
                            target_run_id = runs_data['runs'][-1]['run_id']
                            print(f"Using run_id from {uf}: {target_run_id}")
                            break
            except (json.JSONDecodeError, IndexError, FileNotFoundError):
                continue
    
    if not target_run_id:
        print("Could not determine run_id")
        return None
    
    # Count stats
    user_stats = {}
    
    for user_file in user_files:
        user_file_path = os.path.join(users_dict_path, user_file)
        user_id = os.path.splitext(user_file)[0]
        
        try:
            with open(user_file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            
            # Get actual user_id from file if available
            if len(lines) > 0:
                user_info = json.loads(lines[0].strip())
                if 'user_id' in user_info:
                    user_id = user_info['user_id']
            
            # Check if this file has run data for our target run
            has_run_data = False
            imitation_count = 0
            
            if len(lines) > 2:
                try:
                    runs_content = json.loads(lines[2].strip())
                    target_run = next((r for r in runs_content.get('runs', []) 
                                     if r.get('run_id') == target_run_id), None)
                    if target_run:
                        has_run_data = True
                        imitation_count = len(target_run.get('imitations', []))
                except json.JSONDecodeError:
                    pass
            
            if has_run_data:
                files_with_run_data += 1
                
                # Count UNIQUE rounds (not total evaluations)
                round_count = 0
                total_evaluations = 0
                
                if len(lines) > 3:
                    try:
                        eval_content = json.loads(lines[3].strip())
                        target_evaluations = [e for e in eval_content.get('evaluations', []) 
                                            if e.get('run_id') == target_run_id]
                        
                        total_evaluations = len(target_evaluations)
                        
                        # Count unique rounds
                        unique_rounds = set()
                        for eval_item in target_evaluations:
                            if 'round' in eval_item:
                                unique_rounds.add(eval_item['round'])
                        
                        round_count = len(unique_rounds)
                        
                        # If no 'round' field, assume each evaluation is a round
                        if round_count == 0 and total_evaluations > 0:
                            round_count = total_evaluations
                            
                    except json.JSONDecodeError:
                        pass
                
                user_stats[user_id] = {
                    'rounds': round_count,
                    'imitations': imitation_count,
                    'total_evaluations': total_evaluations  # Added for debugging
                }
        
        except Exception as e:
            print(f"Error processing {user_file}: {e}")
            continue
    
    # Summary
    results = {
        'run_id': target_run_id,
        'total_files_checked': total_files,
        'files_with_run_data': files_with_run_data,
        'total_users_in_run': len(user_stats),
        'user_stats': user_stats
    }
    
    return results

def print_detailed_summary(results):
    """Print a detailed summary including evaluation counts"""
    if not results:
        print("No results to display")
        return
    
    print(f"\n=== DETAILED EXPERIMENT SUMMARY ===")
    print(f"Run ID: {results['run_id']}")
    print(f"Files checked: {results['total_files_checked']}")
    print(f"Files with run data: {results['files_with_run_data']}")
    print(f"Users in this run: {results['total_users_in_run']}")
    
    if results['user_stats']:
        print(f"\nPer-user breakdown:")
        for user_id, stats in results['user_stats'].items():
            print(f"  {user_id}:")
            print(f"    Unique rounds: {stats['rounds']}")
            print(f"    Total evaluations: {stats['total_evaluations']}")
            print(f"    Imitations: {stats['imitations']}")
        
        # Quick stats
        rounds_list = [stats['rounds'] for stats in results['user_stats'].values()]
        imitations_list = [stats['imitations'] for stats in results['user_stats'].values()]
        evaluations_list = [stats['total_evaluations'] for stats in results['user_stats'].values()]
        
        print(f"\nAggregate stats:")
        print(f"  Total unique rounds: {sum(rounds_list)}")
        print(f"  Total evaluations: {sum(evaluations_list)}")
        print(f"  Total imitations: {sum(imitations_list)}")
        
        if rounds_list:
            print(f"  Average rounds per user: {sum(rounds_list)/len(rounds_list):.1f}")
        if evaluations_list:
            print(f"  Average evaluations per user: {sum(evaluations_list)/len(evaluations_list):.1f}")
        if imitations_list:
            print(f"  Average imitations per user: {sum(imitations_list)/len(imitations_list):.1f}")

# Usage example:
if __name__ == "__main__":
    # Replace with your actual path
    path = r"data\filtered_users"
    
    # Count stats for latest run (or specify run_id)
    results = count_experiment_stats(path, '20250708_125947')
    
    if results:
        print_summary(results)


=== EXPERIMENT SUMMARY ===
Run ID: 20250708_125947
Files checked: 161
Files with run data: 3
Users in this run: 3

Per-user breakdown:
  1.5196466670453228e+18: 1 rounds, 9 imitations
  27995424.0: 1 rounds, 9 imitations
  534023.0: 1 rounds, 9 imitations

Quick stats:
  Total rounds across all users: 3
  Total imitations across all users: 27
  Average rounds per user: 1.0
  Average imitations per user: 9.0
